In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Cat_dog/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Cat_dog'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_root])

    def get_the_callback_config(self) -> CallbackConfig:
        config = self.config.save_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])

        prepare_config_callback = CallbackConfig(
            root_dir = config.root_dir,
            tensorboard_root_log_dir = config.tensorboard_root_log_dir,
            checkpoint_model_filepath = config.checkpoint_model_filepath
        )
        return prepare_config_callback

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class PrepareCallback:
    def __init__(self,config:CallbackConfig):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
try:
    config = ConfigurationManager()
    callback_config = config.get_the_callback_config()
    callback = PrepareCallback(config=callback_config)
    callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-09-04 18:08:57,100: INFO: common]: Yaml file: config/config.yaml loaded successfully
[2024-09-04 18:08:57,103: INFO: common]: Yaml file: params.yaml loaded successfully
[2024-09-04 18:08:57,103: INFO: common]: created directory at: data
[2024-09-04 18:08:57,104: INFO: common]: created directory at: data/prepare_callbacks/checkpoint_dir
[2024-09-04 18:08:57,105: INFO: common]: created directory at: data/prepare_callbacks/tensorboard_log_dir


In [11]:
pp = "x/y/z.txt"
os.path.dirname(pp)

'x/y'